In [1]:
import os
import pprint

from langchain_groq import ChatGroq
from langchain_community.document_loaders import WebBaseLoader
from langchain.embeddings import OllamaEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.chains import create_retrieval_chain
from langchain_community.vectorstores import FAISS
import time
import numpy as np
from dotenv import load_dotenv
from langchain_community.vectorstores import Chroma
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_community.embeddings import HuggingFaceBgeEmbeddings


load_dotenv()
groq_api_key=os.environ['GROQ_API_KEY']

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
huggingface_embeddings = HuggingFaceBgeEmbeddings(
    model_name="jaimevera1107/all-MiniLM-L6-v2-similarity-es",
    #model_name="jinaai/jina-embeddings-v2-base-es",
    model_kwargs={'device':'cpu', 'trust_remote_code': True}, 
    encode_kwargs={'normalize_embeddings': False, 'attn_implementation': "eager"},
)

In [3]:
import chromadb
from chromadb.utils import embedding_functions

# Use similarity searching algorithm and return 3 most relevant documents.
client = chromadb.PersistentClient('./db_subvenciones')

db = Chroma(client=client, collection_name="subvenciones",embedding_function=huggingface_embeddings)

#retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 10})

/tmp/ipykernel_376438/477159031.py:7: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db = Chroma(client=client, collection_name="subvenciones",embedding_function=huggingface_embeddings)


In [4]:
collection = client.get_collection("subvenciones")

# query = """   Ayuntamiento de Madrid   """  
#          # Sample question, change to other questions you are interested in.
# # Ejemplo de búsqueda

# results = collection.query(
#     query_embeddings = np.array(huggingface_embeddings.embed_query(query)),
#     #query_texts = [query],

    
    
#     n_results=5,
#     )


In [19]:
#llm=ChatGroq(groq_api_key=groq_api_key,
#model_name="gemma2-9b-it")
from langchain_community.llms import Ollama
#llm = Ollama(model="gemma2:2b", base_url="http://127.0.0.1:11434")
llm=ChatGroq(groq_api_key=groq_api_key,
model_name="llama3-8b-8192")

In [6]:
""" #MULTIQUERY RETRIEVER

from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_openai import ChatOpenAI

multiquery_retriever = MultiQueryRetriever.from_llm(
    retriever=db.as_retriever(search_kwargs={"k": 6}), llm=llm
) """

' #MULTIQUERY RETRIEVER\n\nfrom langchain.retrievers.multi_query import MultiQueryRetriever\nfrom langchain_openai import ChatOpenAI\n\nmultiquery_retriever = MultiQueryRetriever.from_llm(\n    retriever=db.as_retriever(search_kwargs={"k": 6}), llm=llm\n) '

In [7]:
# Set logging for the queries
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [8]:

from langchain.chains.query_constructor.base import AttributeInfo

metadata_field_info = [
    AttributeInfo(
        name="Destinatarios",
        description="Empresas, organizaciones o personas que reciben la ayuda",
        type="string",
    ),
    AttributeInfo(
        name="Organismo",
        description="Organismo que da la ayuda (Ayuntamiento, Consejería, etc.)",
        type="string",
    ),
    AttributeInfo(
        name="Referencia",
        description="Número de referencia y título de la convocatoria (breve descripción)",
        type="string",
    ),
    AttributeInfo(
        name="Sector", 
        description="Sector al que se destina la convocatoria, puede ser uno de estos valores: "+
        "[Turismo,Investigación y desarrollo,Artesano,Comercio,Agroalimentario,Industria,Cultura y comunicación,"+
        "Agrario,Pesquero]", 
        type="string"
    ),
    AttributeInfo(
        name="Subsector", 
        description="Subsector dentro del Sector, completa la información del atributo Sector.", 
        type="string"
    ),
    AttributeInfo(
        name="Tipo", 
        description="Tipo de ayudas", 
        type="string"
    ),
    AttributeInfo(
        name="Ámbito Geográfico", 
        description="Define en qué territorio (ciudad, comunidad o región), se concede la ayuda.", 
        type="string"
    )    
]

In [9]:

from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever

selfqueryRetriever = SelfQueryRetriever.from_llm(
    llm,
    db,
    "Subvenciones y ayudas",
    metadata_field_info
)

In [10]:
#docs = selfqueryRetriever.invoke("Qué subvenciones da el Ayuntamiento de Cilleros")
#pprint.pp(docs)

In [11]:
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 20})

In [12]:
# docs = retriever.invoke("Consejería de Economía, Empleo y Transformación Digital")
# pprint.pp(docs)

In [13]:
from langchain.retrievers import MergerRetriever

mergeRetriever = MergerRetriever(retrievers=[selfqueryRetriever, retriever])



In [14]:
# docs = mergeRetriever.invoke("Extremadura")
# pprint.pp(docs)

In [15]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder

#Reranker, to select the best 5 documents

model = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-v2-m3")
compressor = CrossEncoderReranker(model=model, top_n=5)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=mergeRetriever
)

# compressed_docs = compression_retriever.invoke("Referencia 115300")
# pprint.pp(compressed_docs)

In [22]:
from langchain.chains import ReduceDocumentsChain

qa_template = """Eres un asistente para responder a preguntas en español "
    " basándote en los documentos proporcionados más abajo. "
    "Debes reproducir exactamente el fragmento de texto donde viene la respuesta"
    "Ordena todas las respuestas que encuentres en diferentes líneas."
    "Traduce la respuesta al español antes de mostrarla.
    "Si algún documento no contiene la respuesta, ignóralo."
    "\n\n"
    "{context}"

Pregunta: {question}
Respuesta:"""

prompt = PromptTemplate(template=qa_template,
                            input_variables=['context','question'])
combine_custom_prompt='''
Responde con todas las respuestas que encuentres en diferentes documentos.

Text:`{context}`
'''
""" 
""" 
combine_prompt_template = PromptTemplate(
    template=combine_custom_prompt, 
    input_variables=['context']
)
qa_chain = RetrievalQA.from_chain_type(llm, retriever=compression_retriever, chain_type="map_reduce",return_source_documents=True,
 chain_type_kwargs= {
        "verbose": False,
        "question_prompt": prompt,
        "combine_prompt": combine_prompt_template,
        "combine_document_variable_name": "context"})

""" qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                          chain_type="stuff",
                                          retriever=compression_retriever,
                                          return_source_documents=True) """


query = """ ¿Qué subvenciones se dan en Cantabria?   """  
question = query

result=qa_chain.invoke(question)
pprint.pp(result)

{'query': ' ¿Qué subvenciones se dan en Cantabria?   ',
 'result': 'La respuesta a la pregunta "¿Qué subvenciones se dan en '
           'Cantabria?" se encuentra en el punto 1 del extracto de la '
           'resolución de la consejera de Desarrollo Rural, Ganadería, Pesca y '
           'Alimentación de 28 de octubre de 2024. Según este punto, se '
           'convocan ayudas para la promoción de productos alimentarios de '
           'Cantabria, específicamente:\n'
           '\n'
           '* Ayudas a la organización de ferias y mercados alimentarios.\n'
           '* Ayudas a la asistencia y participación a ferias alimentarias.\n'
           '\n'
           'La respuesta traducida al español es:\n'
           '\n'
           '"Se convocan ayudas para la promoción de productos alimentarios de '
           'Cantabria, específicamente: ayudas a la organización de ferias y '
           'mercados alimentarios y ayudas a la asistencia y participación a '
           'ferias alimentarias

In [ ]:
relevant_docs = result['source_documents']
print(f'There are {len(relevant_docs)} documents retrieved which are relevant to the query.')
print("*" * 100)
for i, doc in enumerate(relevant_docs):
    print(f"Relevant Document #{i+1}:\nSource file: {doc.metadata['source']}, Page: {doc.metadata['page']}\nContent: {doc.page_content}")
    print("-"*100)
    print(f'There are {len(relevant_docs)} documents retrieved which are relevant to the query.')

In [ ]:
print (relevant_docs)